In [ ]:
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve,auc,classification_report,make_scorer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from link_prediction import *

In [ ]:
#Load the entity embeddings dict
with open("dbpedia50_transe_entity_embeddings.pkl", "rb") as f:
    entity_embeddings = pickle.load(f)
#Load the predicate embeddings dict
with open("dbpedia50_transe_predicate_embeddings.pkl", "rb") as f:
    predicate_embeddings = pickle.load(f)

train_triples = pd.read_csv('dbpedia50_train.csv', dtype=str)
test_triples = pd.read_csv('dbpedia50_test_filtered.csv', dtype=str)

**Create Negative Samples**

In [4]:
training_object=Training(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='TransE')
tail_df = training_object.create_training_data_filtered(n=1, creating_for="tail")
head_df = training_object.create_training_data_filtered(n=1, creating_for="head")
print(len(tail_df))
print(len(head_df))

Processing triples: 100%|██████████| 32388/32388 [00:42<00:00, 758.48triple/s] 

29346
11854


**Prepare Training Data**

In [5]:
train_triples['label']=1
train_df = pd.concat([head_df,tail_df,train_triples], axis=0)
# Reset the index
train_df.reset_index(drop=True, inplace=True)
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)
# Step 1: Prepare the training data
train_df['embedding'] = train_df.apply(lambda row: training_object.get_embedding(row).cpu().numpy(), axis=1)
X_train = np.vstack(train_df['embedding'].values)  # Stack embeddings into a matrix
y_train = train_df['label'].values  # Labels
# Step 1: Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_df['embedding'].values),  # Embeddings matrix
    train_df['label'].values,                # Labels
    test_size=0.1,                           # 10% of the data for testing
    random_state=42,                         # For reproducibility
    stratify=train_df['label'].values        # Maintain label distribution
)
train_df['label'].value_counts()

,count
label,
0,40018
1,32203


**Set Models**

**MLP**

In [6]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [7]:
param_grid_mlp = {
    'hidden_layer_sizes': [(32, 32), (64, 64), (128, 128), (256, 256)],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [25, 50, 100]
}


results_mlp = []
for hidden_layer_sizes in param_grid_mlp['hidden_layer_sizes']:
    for alpha in param_grid_mlp['alpha']:
        for learning_rate_init in param_grid_mlp['learning_rate_init']:
            for max_iter in param_grid_mlp['max_iter']:
                mlp = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    alpha=alpha,
                    learning_rate_init=learning_rate_init,
                    max_iter=max_iter,
                    solver='adam',
                    early_stopping=True,
                    batch_size=32,
                    random_state=42
                )
                mlp.fit(X_train, y_train)
                preds = mlp.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_mlp.append({
                    'hidden_layer_sizes': hidden_layer_sizes,
                    'alpha': alpha,
                    'learning_rate_init': learning_rate_init,
                    'max_iter': max_iter,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_mlp = pd.DataFrame(results_mlp)

best_mlp_params = df_results_mlp.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for MLP:", best_mlp_params)

Best parameters for MLP: {'hidden_layer_sizes': (256, 256), 'alpha': 0.01, 'learning_rate_init': 0.001, 'max_iter': 100, 'f1_score': 0.8302895322939866, 'accuracy': 0.8417555032534958, 'precision_0': 0.8854138581827986, 'precision_1': 0.7956744450768355, 'recall_0': 0.8205897051474262, 'recall_1': 0.8680533995653523}


**XGB**

In [8]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    # 'scale_pos_weight': [0.5, 1, 2],  # commented out as provided
    'learning_rate': [0.01, 0.1, 0.2]
}


results_xgb = []
for n_estimators in param_grid_xgb['n_estimators']:
    for max_depth in param_grid_xgb['max_depth']:
        for learning_rate in param_grid_xgb['learning_rate']:
            xgb = XGBClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                random_state=42,
            )
            xgb.fit(X_train, y_train)
            preds = xgb.predict(X_test)

            # Calculate metrics
            score_f1 = f1_score(y_test, preds)
            acc = accuracy_score(y_test, preds)
            precisions = precision_score(y_test, preds, average=None, zero_division=0)
            recalls = recall_score(y_test, preds, average=None, zero_division=0)

            results_xgb.append({
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'f1_score': score_f1,
                'accuracy': acc,
                'precision_0': precisions[0],
                'precision_1': precisions[1],
                'recall_0': recalls[0],
                'recall_1': recalls[1]
            })

df_results_xgb = pd.DataFrame(results_xgb)
best_xgb_params = df_results_xgb.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for XGBClassifier:", best_xgb_params)

Best parameters for XGBClassifier: {'n_estimators': 300.0, 'max_depth': 5.0, 'learning_rate': 0.2, 'f1_score': 0.7742037212235888, 'accuracy': 0.8017444275231899, 'precision_0': 0.8132618235007314, 'precision_1': 0.7866068567766742, 'recall_0': 0.8335832083958021, 'recall_1': 0.7621856566283762}


**LGBM**

In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2],
    'is_unbalance': [True, False]
}

results_lgbm = []
for n_estimators in param_grid_lgbm['n_estimators']:
    for max_depth in param_grid_lgbm['max_depth']:
        for learning_rate in param_grid_lgbm['learning_rate']:
            for is_unbalance in param_grid_lgbm['is_unbalance']:
                lgbm = LGBMClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    is_unbalance=is_unbalance,
                    random_state=42
                )
                lgbm.fit(X_train, y_train)
                preds = lgbm.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_lgbm.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'is_unbalance': is_unbalance,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_lgbm = pd.DataFrame(results_lgbm)
best_lgbm_params = df_results_lgbm.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for LGBM:", best_lgbm_params)

**Train Ensemble**

In [ ]:
# Best MLPClassifier
best_mlp = MLPClassifier(
    hidden_layer_sizes=best_mlp_params['hidden_layer_sizes'],
    alpha=best_mlp_params['alpha'],
    learning_rate_init=best_mlp_params['learning_rate_init'],
    max_iter=best_mlp_params['max_iter'],
    solver='adam',
    early_stopping=True,
    batch_size=32,
    random_state=42
)
best_mlp.fit(X_train, y_train)
preds_mlp = best_mlp.predict(X_test)
report_mlp = classification_report(y_test, preds_mlp)

# Best XGBClassifier
best_xgb = XGBClassifier(
    n_estimators=int(best_xgb_params['n_estimators']),
    max_depth=int(best_xgb_params['max_depth']),
    learning_rate=float(best_xgb_params['learning_rate']),
    random_state=42,
)
best_xgb.fit(X_train, y_train)
preds_xgb = best_xgb.predict(X_test)
report_xgb = classification_report(y_test, preds_xgb)

# Best LGBMClassifier
best_lgbm = LGBMClassifier(
    n_estimators=best_lgbm_params['n_estimators'],
    max_depth=best_lgbm_params['max_depth'],
    learning_rate=best_lgbm_params['learning_rate'],
    is_unbalance=best_lgbm_params['is_unbalance'],
    random_state=42
)
best_lgbm.fit(X_train, y_train)
preds_lgbm = best_lgbm.predict(X_test)
report_lgbm = classification_report(y_test, preds_lgbm)

######################################
# Ensemble the best models using VotingClassifier
######################################
ensemble_model = VotingClassifier(
    estimators=[
        ('mlp', best_mlp),
        ('xgb', best_xgb),
        ('lgbm', best_lgbm)
    ],
    voting='soft',  # Soft voting uses predicted probabilities
    n_jobs=-1
)

start_time = time.time()
ensemble_model.fit(X_train, y_train)
end_time = time.time()

ensemble_time = end_time - start_time
preds_ensemble = ensemble_model.predict(X_test)
ensemble_acc = accuracy_score(y_test, preds_ensemble)
report_ensemble = classification_report(y_test, preds_ensemble)

In [18]:
preds_ensemble = ensemble_model.predict(X_test)
print("Ensemble Accuracy:", accuracy_score(y_test, preds_ensemble))
print("Ensemble Classification Report:\n", classification_report(y_test, preds_ensemble))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Ensemble Accuracy: 0.8520005537865153
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.87      4002
           1       0.83      0.84      0.83      3221

    accuracy                           0.85      7223
   macro avg       0.85      0.85      0.85      7223
weighted avg       0.85      0.85      0.85      7223



In [ ]:
with open("/content/drive/MyDrive/link_prediction/tuning/dbpedia50_transe_tuning.txt", "w") as f:
    f.write("##############################\n")
    f.write("MLPClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_mlp.to_string())
    f.write("\n\nBest Parameters for MLPClassifier:\n")
    f.write(str(best_mlp_params))
    f.write("\n\nClassification Report for Best MLPClassifier:\n")
    f.write(report_mlp)

    f.write("\n\n\n##############################\n")
    f.write("XGBClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_xgb.to_string())
    f.write("\n\nBest Parameters for XGBClassifier:\n")
    f.write(str(best_xgb_params))
    f.write("\n\nClassification Report for Best XGBClassifier:\n")
    f.write(report_xgb)

    f.write("\n\n\n##############################\n")
    f.write("LGBMClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_lgbm.to_string())
    f.write("\n\nBest Parameters for LGBMClassifier:\n")
    f.write(str(best_lgbm_params))
    f.write("\n\nClassification Report for Best LGBMClassifier:\n")
    f.write(report_lgbm)

    f.write("\n\n\n##############################\n")
    f.write("Ensemble Model Performance\n")
    f.write("##############################\n")
    f.write("Ensemble Accuracy: {:.4f}\n".format(ensemble_acc))
    f.write("Ensemble Classification Report:\n")
    f.write(report_ensemble)

print("Results saved to model_results.txt")

Results saved to model_results.txt


**Rerank Hitk**

In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
tripleEvaluator=TripleEvaluator(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='TransE', k=50)

Precomputing top-k predictions: 100%|██████████| 2098/2098 [00:23<00:00, 88.16it/s]


In [22]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10,20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19,20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 15.65% | tail: 35.86% | overall: 27.91%
k=1 | th=2 | head: 19.08% | tail: 38.86% | overall: 31.08%
k=1 | th=3 | head: 18.26% | tail: 38.57% | overall: 30.58%
k=1 | th=4 | head: 18.11% | tail: 37.41% | overall: 29.82%
k=1 | th=5 | head: 17.51% | tail: 37.36% | overall: 29.55%
k=1 | th=6 | head: 17.59% | tail: 36.64% | overall: 29.14%
k=1 | th=7 | head: 17.66% | tail: 36.64% | overall: 29.17%
k=1 | th=8 | head: 17.73% | tail: 36.54% | overall: 29.14%
k=1 | th=9 | head: 17.81% | tail: 36.35% | overall: 29.05%
k=1 | th=10 | head: 17.73% | tail: 35.81% | overall: 28.70%
k=1 | th=20 | head: 17.06% | tail: 34.36% | overall: 27.56%
k=1 | th=30 | head: 16.84% | tail: 33.74% | overall: 27.09%
k=1 | th=40 | head: 16.47% | tail: 33.06% | overall: 26.53%
k=1 | th=50 | head: 16.47% | tail: 32.53% | overall: 26.21%
k=3 | th=3 | head: 43.82% | tail: 58.53% | overall: 52.74%
k=3 | th=4 | head: 43.07% | tail: 58.72% | overall: 52.57%
k=3 | th=5 | head: 42.32% | tail: 58.39% | overall:

In [23]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 15.65% | tail: 35.86% | overall: 27.91%
k=1 | th=2 | head: 20.04% | tail: 40.99% | overall: 32.75%
k=1 | th=3 | head: 19.67% | tail: 41.52% | overall: 32.92%
k=1 | th=4 | head: 18.70% | tail: 40.79% | overall: 32.10%
k=1 | th=5 | head: 18.78% | tail: 40.89% | overall: 32.19%
k=1 | th=6 | head: 18.48% | tail: 40.94% | overall: 32.10%
k=1 | th=7 | head: 18.48% | tail: 41.18% | overall: 32.25%
k=1 | th=8 | head: 18.33% | tail: 40.84% | overall: 31.98%
k=1 | th=9 | head: 18.18% | tail: 40.45% | overall: 31.69%
k=1 | th=10 | head: 18.11% | tail: 40.45% | overall: 31.66%
k=1 | th=20 | head: 17.51% | tail: 40.50% | overall: 31.46%
k=1 | th=30 | head: 17.44% | tail: 40.31% | overall: 31.31%
k=1 | th=40 | head: 17.21% | tail: 40.12% | overall: 31.11%
k=1 | th=50 | head: 17.06% | tail: 39.73% | overall: 30.81%
k=3 | th=3 | head: 43.82% | tail: 58.53% | overall: 52.74%
k=3 | th=4 | head: 42.62% | tail: 59.21% | overall: 52.68%
k=3 | th=5 | head: 41.58% | tail: 59.30% | overall:

In [24]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 15.65% | tail: 35.86% | overall: 27.91%
k=1 | th=2 | head: 19.15% | tail: 40.84% | overall: 32.31%
k=1 | th=3 | head: 18.63% | tail: 41.08% | overall: 32.25%
k=1 | th=4 | head: 18.18% | tail: 40.31% | overall: 31.60%
k=1 | th=5 | head: 17.96% | tail: 40.36% | overall: 31.55%
k=1 | th=6 | head: 17.73% | tail: 40.21% | overall: 31.37%
k=1 | th=7 | head: 17.59% | tail: 40.12% | overall: 31.25%
k=1 | th=8 | head: 17.59% | tail: 40.02% | overall: 31.19%
k=1 | th=9 | head: 17.06% | tail: 39.87% | overall: 30.90%
k=1 | th=10 | head: 16.99% | tail: 39.54% | overall: 30.67%
k=1 | th=20 | head: 16.10% | tail: 38.76% | overall: 29.84%
k=1 | th=30 | head: 16.32% | tail: 38.28% | overall: 29.64%
k=1 | th=40 | head: 15.87% | tail: 37.94% | overall: 29.26%
k=1 | th=50 | head: 15.80% | tail: 37.70% | overall: 29.08%
k=3 | th=3 | head: 43.82% | tail: 58.53% | overall: 52.74%
k=3 | th=4 | head: 43.00% | tail: 58.77% | overall: 52.57%
k=3 | th=5 | head: 42.47% | tail: 58.92% | overall:

In [25]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 15.65% | tail: 35.86% | overall: 27.91%
k=1 | th=2 | head: 19.30% | tail: 40.02% | overall: 31.87%
k=1 | th=3 | head: 19.82% | tail: 40.74% | overall: 32.51%
k=1 | th=4 | head: 19.37% | tail: 40.55% | overall: 32.22%
k=1 | th=5 | head: 19.00% | tail: 41.08% | overall: 32.40%
k=1 | th=6 | head: 18.78% | tail: 41.08% | overall: 32.31%
k=1 | th=7 | head: 18.63% | tail: 41.28% | overall: 32.37%
k=1 | th=8 | head: 18.63% | tail: 41.32% | overall: 32.40%
k=1 | th=9 | head: 18.70% | tail: 41.28% | overall: 32.40%
k=1 | th=10 | head: 18.78% | tail: 41.28% | overall: 32.42%
k=1 | th=20 | head: 18.26% | tail: 41.32% | overall: 32.25%
k=1 | th=30 | head: 18.48% | tail: 41.28% | overall: 32.31%
k=1 | th=40 | head: 18.33% | tail: 41.23% | overall: 32.22%
k=1 | th=50 | head: 18.33% | tail: 41.23% | overall: 32.22%
k=3 | th=3 | head: 43.82% | tail: 58.53% | overall: 52.74%
k=3 | th=4 | head: 43.67% | tail: 59.30% | overall: 53.15%
k=3 | th=5 | head: 43.89% | tail: 59.64% | overall: